In [1]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np

from torchvision import transforms


from PIL import Image
from torch import optim,nn


from torch.utils.data.dataset import Dataset

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet,self).__init__()
        # convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=12,kernel_size=5,stride=1,padding=0)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)
        
        self.conv2 = nn.Conv2d(in_channels=12,out_channels=24,kernel_size=3,stride=1,padding=0)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)

        self.conv3 = nn.Conv2d(in_channels=24,out_channels=32,kernel_size=2,stride=1,padding=0)
        self.linear1 = nn.Linear(in_features = 1568, out_features = 108)
        self.linear2 = nn.Linear(in_features = 108, out_features = 32)
        self.linear3 = nn.Linear(in_features = 32, out_features = 2) # Binary Classification 
        
    def forward(self, x):
        Y = F.relu(self.conv3(self.maxpool2(F.relu(self.conv2(self.maxpool1(F.relu(self.conv1(x)))))))).reshape(x.shape[0],1,1,-1)
        Y = F.relu(self.linear3(F.relu(self.linear2(F.relu(self.linear1(Y))))))
        Y = Y.view(Y.shape[0],2)
        return Y
       



In [4]:

class Accumulator: 
  def __init__(self, n):
    self.data = [0.0] * n
  def add(self, *args):
    self.data = [a + float(b) for a, b in zip(self.data, args)]
  def reset(self):
    self.data = [0.0] * len(self.data)
  def __getitem__(self, idx):
    return self.data[idx]






def accuracy(y_hat, y): 
  if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
    y_hat = y_hat.argmax(axis=1)
  cmp = y_hat.type(y.dtype) == y
  return float(cmp.type(y.dtype).sum())





In [5]:

def train(net,train_loader,optimizer,n_epochs,loss_function):
    net = net.to(device)
    for epoch in range(n_epochs): 
        metric = Accumulator(4)
        for step, (inputs, labels) in enumerate(train_loader, start=0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = inputs.to(device), labels.to(device) 
            net.train()
            optimizer.zero_grad()
            pred = net(inputs)
            l = loss_function(pred, labels)
            l.sum().backward()
            optimizer.step()
            loss = l.sum()
            acc = accuracy(pred, labels)
            metric.add(loss, acc, labels.shape[0], labels.numel())
        loss = metric[0] / metric[2]
        acc = metric[1] / metric[3]
              
        print('Train Epoch: {}/{} \tLoss: {:.6f}\tacc: {:.6f}'.format(epoch, n_epochs, loss, acc))

    print('Finished Training')
    save_path = './pretrained/MyNet.pth'
    torch.save(net.state_dict(), save_path)


In [6]:
def test(net, test_loader, loss_function):
    net.eval()
    test_loss = 0
    metric = Accumulator(3)
    with torch.no_grad():
      for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        pred = net(inputs)
        metric.add(loss_function(pred, labels).sum(), accuracy(pred, labels), labels.numel())    
    test_loss = metric[0] / metric[2]
    acc = metric[1] / metric[2]
    print("Test set: Average loss: {:.4f}\t Acc {:.2f}".format(test_loss, acc))

In [7]:

class DatasetFromCSV(Dataset):
    def __init__(self, csv_path, height=16, width=16, transforms=None):
        self.data = pd.read_csv(csv_path)
        self.labels = np.asarray(self.data.iloc[:, 256])
        self.height = height
        self.width = width
        self.transforms = transforms
 
    def __getitem__(self, index):
        single_image_label = self.labels[index]
        # 1D array ([256]) reshape -> 2D array ([16,16])
        img_as_np = np.asarray(self.data.iloc[index][0:256]).reshape(16, 16, 1).astype(float)
        
        # transform numpy to tensor
        if self.transforms is not None:
            img_as_tensor = self.transforms(img_as_np)
            img_as_tensor = img_as_tensor.type(torch.FloatTensor)

        return (img_as_tensor, single_image_label)
 
    def __len__(self):
        return len(self.data.index)


In [8]:
if __name__ == '__main__':
    n_epochs = 80
    train_batch_size = 32
    test_batch_size = 32
    learning_rate = 5e-4
    
    transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))]) # normalization
    
    train_data= DatasetFromCSV('trainingdata.csv',16,16,transform)
    test_data = DatasetFromCSV("testingdata.csv",16,16,transform)
 
    train_loader = torch.utils.data.DataLoader(train_data,batch_size=train_batch_size, shuffle=True, num_workers=0)
    test_loader = torch.utils.data.DataLoader(test_data,batch_size=test_batch_size, shuffle=True, num_workers=0)

    
    net = MyNet()
    # optimizer 
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate,weight_decay=0.001)
    loss_function = nn.CrossEntropyLoss(reduction="none")
    train(net,train_loader,optimizer,n_epochs,loss_function)
    test(net,test_loader,loss_function)

Train Epoch: 0/80 	Loss: 0.692104	acc: 0.533724
Train Epoch: 1/80 	Loss: 0.686764	acc: 0.593353
Train Epoch: 2/80 	Loss: 0.682426	acc: 0.593353
Train Epoch: 3/80 	Loss: 0.679683	acc: 0.593353
Train Epoch: 4/80 	Loss: 0.677851	acc: 0.593353
Train Epoch: 5/80 	Loss: 0.676713	acc: 0.593353
Train Epoch: 6/80 	Loss: 0.676110	acc: 0.593353
Train Epoch: 7/80 	Loss: 0.675669	acc: 0.593353
Train Epoch: 8/80 	Loss: 0.675411	acc: 0.593353
Train Epoch: 9/80 	Loss: 0.675250	acc: 0.593353
Train Epoch: 10/80 	Loss: 0.675105	acc: 0.593353
Train Epoch: 11/80 	Loss: 0.674834	acc: 0.593353
Train Epoch: 12/80 	Loss: 0.674734	acc: 0.593353
Train Epoch: 13/80 	Loss: 0.674650	acc: 0.593353
Train Epoch: 14/80 	Loss: 0.674495	acc: 0.593353
Train Epoch: 15/80 	Loss: 0.674318	acc: 0.593353
Train Epoch: 16/80 	Loss: 0.674301	acc: 0.593353
Train Epoch: 17/80 	Loss: 0.673973	acc: 0.593353
Train Epoch: 18/80 	Loss: 0.673931	acc: 0.593353
Train Epoch: 19/80 	Loss: 0.673767	acc: 0.593353
Train Epoch: 20/80 	Loss: 0.67